## import

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import os

import nibabel as nib
from nilearn.datasets import fetch_surf_fsaverage

import brainstat
from brainstat.stats.terms import FixedEffect

import matplotlib.pyplot as plt
from scipy import io

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69

from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

import scipy.io
import re

import matplotlib

In [ ]:
import sys , numpy as np
mod = sys.modules[__name__]

## make model

In [ ]:
brain = pd.read_excel("./data/환자군_정상군_181104_결측값업데이트_scores2_bypark.xlsx")

In [ ]:
ha_freq_sbt = brain[np.array(('Sex','Age','HA_freq_m_fMRI'))]

In [ ]:
ha_freq_sbt = ha_freq_sbt.dropna()

In [ ]:
age = ha_freq_sbt['Age']
sex = ha_freq_sbt['Sex']
sex=np.array(sex)
sex=sex.reshape(sex.shape[0],1)
ha_freq = ha_freq_sbt['HA_freq_m_fMRI']
NumSubj = np.size(ha_freq)

In [ ]:
# # make sex and grp as dummy variables
sex_dum1 = np.zeros((NumSubj,1))
sex_dum1 = np.where(sex==0, 1, sex_dum1)
sex_dum2 = np.zeros((NumSubj,1))
sex_dum2 = np.where(sex==1, 1, sex_dum2)
sex_dum = np.concatenate((sex_dum1, sex_dum2), axis=1) # F,M

In [ ]:
sex_dum = pd.DataFrame(sex_dum)
sex_dum.columns = ['F','M']

In [ ]:
intercept = np.ones((NumSubj,1))

In [ ]:
intercept = pd.DataFrame(intercept)
intercept.columns = ['intercept']

In [ ]:
age = age.reset_index(drop=True)
ha_freq = ha_freq.reset_index(drop=True)

In [ ]:
rm = pd.concat([intercept,sex_dum,age],axis=1)

In [ ]:
# reduced model with sex&age terms
rm

### <span style="color:#2D3748; background-color:#fff5b1;">make full model with sex&age terms, cortical eigenvector&subcortical weighted manifold terms</span>

### load cortex files

In [ ]:
from glob import glob
path = './data/schaefer200/cortex/'
filepaths = glob(path+'*.csv') # load path of csv files

In [ ]:
import pandas as pd
for i in range(len(filepaths)):
    dat = pd.read_csv(filepaths[i], header=None)
    res = re.split('[\\\/]',filepaths[i])
    globals()[res[-1][:-4]+'_a'] = dat.to_numpy() # load csv files as array type

In [ ]:
for i in range(1,100+1):
    globals()['ctx{}'.format(i)] = getattr(mod,'g{}_200_a'.format(i))

### make cortico-cortical connectivity matrix

In [ ]:
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure 
correlation_measure = ConnectivityMeasure(kind='correlation')

for i in range(1,100+1):
    globals()['ctx{}_c'.format(i)] = correlation_measure.fit_transform([getattr(mod, 'ctx{}'.format(i))]) 

In [ ]:
for i in range(1, 100+1):
    globals()['ctx{}_c'.format(i)] = getattr(mod, 'ctx{}_c'.format(i))[0]
# (1,200,200) array shape -> (200,200) array shape

### 1. z-transformation
### 2. transform all diagonals into zeros

In [ ]:
tab = np.zeros(shape=(200,200))

In [ ]:
for i in range(1,101):
    tab = np.arctanh(getattr(mod, 'ctx{}_c'.format(i)))
    np.fill_diagonal(tab, np.zeros(200))
    globals()['ctx{}_cz'.format(i)] = tab

### get group connectivity matrix averaging z-transformed 100 connectivity matrix

In [ ]:
import numpy
ctx_sum = numpy.zeros(shape=(200,200))
for i in range(1,101):
    ctx_sum = ctx_sum + getattr(mod, 'ctx{}_cz'.format(i))

ctx_mean = ctx_sum/100 

### make template of eigenvectors 1,2 and 3 with hcp

In [ ]:
mat_file_name = "./data/gradients_FC.mat"
mat_file = scipy.io.loadmat(mat_file_name)

In [ ]:
gm_mean = mat_file['gm_mean']

### align group connectivity matrix with hcp eigenvectors

In [ ]:
# Gradient alignment
align = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)

align.fit(ctx_mean, reference = gm_mean)

### align each individual connectivity matrix with hcp-aligned group connectivity matrix

> we call it individual eigenvector

In [ ]:
for i in range(1,100+1):
    align2 = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)
    align2.fit(getattr(mod,'ctx{}_cz'.format(i)), reference = align.aligned_)
    globals()['grad_{}'.format(i)] = align2.aligned_[:,0:2+1]

we reset ha_freq_sbt to select eigenvectors corresponding to indices of ha_freq_sbt

In [ ]:
ha_freq_sbt = brain[np.array(('Sex','Age','HA_freq_m_fMRI'))]
ha_freq_sbt = ha_freq_sbt.dropna()

In [ ]:
Z = np.dstack([grad_1,grad_2])

In [ ]:
for i in ha_freq_sbt.index[2:]:
    Z = np.dstack([Z, getattr(mod, 'grad_{}'.format(i))])

In [ ]:
data = np.zeros((NumSubj, 200, 3))
for ns in range(0,NumSubj):
    mat = Z[:,0:3,ns]
    data[ns,:,:] = mat

In [ ]:
name1 = ['gra1_'+str(i+1) for i in range(0,199+1)]

In [ ]:
name2 = ['gra2_'+str(i+1) for i in range(0,199+1)]

In [ ]:
name3 = ['gra3_'+str(i+1) for i in range(0,199+1)]

In [ ]:
dat1 = pd.DataFrame(data[:,:,0], columns=name1)

In [ ]:
dat2 = pd.DataFrame(data[:,:,1], columns=name2)

In [ ]:
dat3 = pd.DataFrame(data[:,:,2], columns=name3)

In [ ]:
# full model
fm = pd.concat([rm,dat1,dat2,dat3],axis=1)

In [ ]:
fm

### regress out

In [ ]:
#Load libraries
from sklearn.datasets import load_wine
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

before regressing out, we compile eigenvectors

In [ ]:
X_dat = pd.concat([dat1,dat2,dat3],axis=1)

1. we make linear model which has X = rm and y = one column among all features
2. we then get predicted value by putting X into the model again

> lm.fit(X, y)

In [ ]:
# 200 nodes of gradient 1
for i in range(0,199+1):
    lm = LinearRegression()
    lm.fit(rm, dat1.iloc[:,i])
    pred = lm.predict(rm)
    globals()['gra1_{}'.format(i+1)] = pred.reshape(NumSubj,1)

In [ ]:
# 200 nodes of gradient 2
for i in range(0,199+1):
    lm = LinearRegression()
    lm.fit(rm, dat2.iloc[:,i])
    pred = lm.predict(rm)
    globals()['gra2_{}'.format(i+1)] = pred.reshape(NumSubj,1)

In [ ]:
# 200 nodes of gradient 3
for i in range(0,199+1):
    lm = LinearRegression()
    lm.fit(rm, dat3.iloc[:,i])
    pred = lm.predict(rm)
    globals()['gra3_{}'.format(i+1)] = pred.reshape(NumSubj,1)

we now merge these 600 terms

In [ ]:
grad1 = gra1_1
for i in range(2,200+1):
    tmp = getattr(mod,'gra1_{}'.format(i))
    grad1 = np.concatenate((grad1,tmp),axis=1)

In [ ]:
grad2 = gra2_1
for i in range(2,200+1):
    tmp = getattr(mod,'gra2_{}'.format(i))
    grad2 = np.concatenate((grad2,tmp),axis=1)

In [ ]:
grad3 = gra3_1
for i in range(2,200+1):
    tmp = getattr(mod,'gra3_{}'.format(i))
    grad3 = np.concatenate((grad3,tmp),axis=1)

In [ ]:
grad = np.concatenate([grad1,grad2,grad3],axis=1)
grad = pd.DataFrame(grad,columns = X_dat.columns)

1. we remove X_dat from grad to get corrected terms

In [ ]:
X_grad = X_dat - grad

In [ ]:
X_grad

2. we then add intercept term to them

In [ ]:
X_grad = pd.concat([intercept,X_grad],axis=1)

### subcortical part

In [ ]:
from glob import glob
path = './data/schaefer200/subcortex/'
filepaths = glob(path+'*.csv') # load path of csv files

In [ ]:
for i in range(len(filepaths)):
    dat = pd.read_csv(filepaths[i], header=None)
    res = re.split('[\\\/]',filepaths[i])
    globals()[res[-1][:-4]+'_a'] = dat.to_numpy() # load csv files as numpy type

In [ ]:
for i in range(1,50+1):
    globals()['sctx{}'.format(i)] = getattr(mod, 'gp{}_200_a'.format(i)) 
# we set gp{}_200_a as sctx{}

In [ ]:
for i in range(1,50+1):
    globals()['sctx{}'.format(str(i+50))] = getattr(mod, 'gn{}_200_a'.format(i)) 
# we set gn{}_200_a as sctx{}

### make subcortico-subcortical connectivity matrix

In [ ]:
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure 
correlation_measure = ConnectivityMeasure(kind='correlation')

for i in range(1,100+1):
    globals()['sctx{}_c'.format(i)] = correlation_measure.fit_transform([getattr(mod, 'sctx{}'.format(i))]) 

In [ ]:
for i in range(1, 100+1):
    globals()['sctx{}_c'.format(i)] = getattr(mod, 'sctx{}_c'.format(i))[0]
# (1,17,17) array shape -> (17,17) array shape

### 1. z-transformation
### 2. transform all diagonals into zeros

In [ ]:
tab = np.zeros(shape=(17,17))

In [ ]:
for i in range(1,101):
    tab = np.arctanh(getattr(mod, 'sctx{}_c'.format(i)))
    np.fill_diagonal(tab, np.zeros(17))
    globals()['sctx{}_cz'.format(i)] = tab

### get group connectivity matrix averaging z-transformed 100 connectivity matrix

In [ ]:
import numpy
sctx_sum = numpy.zeros(shape=(17,17))
for i in range(1,101):
    sctx_sum = sctx_sum + getattr(mod, 'sctx{}_cz'.format(i))

In [ ]:
sctx_mean = sctx_sum/100

### make subcortico-cortical connectivity matrix

In [ ]:
for i in range(1,100+1):
    print('\n\033[1m'+'##{}th person'.format(i))
    npz = np.zeros((200,17))
    x = getattr(mod,'ctx{}'.format(i))
    y = getattr(mod,'sctx{}'.format(i))
    for j in range(0,199+1):
        print('\n\033[1m'+'{}th row'.format(j+1))
        for k in range(0,16+1):
            print('\033[0m' +'{}th col'.format(k+1))
            npz[j][k] = np.corrcoef(x.T[j],y.T[k])[0][1]
            globals()['sc_ctx{}_c'.format(i)] = npz

we make 200x9 matrix averaging left and right regions of 200x17 matrix
> we can get 200x9 matrices of 100 individuals

In [ ]:
for i in range(1,100+1):
    npz = np.zeros((9,200))
    tmp = getattr(mod,'sc_ctx{}_c'.format(i)).T
    for j in range(0,7+1):
        npz[j] = (tmp[j]+tmp[j+8])/2
        npz[8] = tmp[16]
        globals()['sc_ctx{}_c2'.format(i)] = npz

## <span style="color:#2D3748; background-color:#fff5b1;"> make subcortical-weigthed manifold (henceforth eigenvector)</span>

### make template of eigenvectors 1,2 and 3 with hcp

In [ ]:
mat_file_name = "./data/gradients_FC.mat"
mat_file = scipy.io.loadmat(mat_file_name)

In [ ]:
gm_mean = mat_file['gm_mean']

### align group connectivity matrix with hcp eigenvectors

In [ ]:
# Gradient alignment
align = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)

In [ ]:
align.fit(ctx_mean, reference = gm_mean)

### align each individual connectivity matrix with hcp-aligned group connectivity matrix

> we call it individual eigenvector

In [ ]:
for i in range(1,100+1):
    align2 = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)
    align2.fit(getattr(mod,'ctx{}_cz'.format(i)), reference = align.aligned_)
    globals()['grad_{}'.format(i)] = align2.aligned_[:,0:2+1]

### we perform element-wise multiplication with subcortico-cortical connectivity matrix and cortical eigenvectors

In [ ]:
for i in range(1,100+1):
    grad = getattr(mod,'grad_{}'.format(i)).T
    sc_ctx = getattr(mod,'sc_ctx{}_c2'.format(i))
    mat = np.zeros((3,9,200))
    for j in range(0,2+1):
        for k in range(0,8+1):
            mat[j,k,:] = np.multiply(grad[j],sc_ctx[k]) 
    globals()['sw_manifold{}'.format(i)] = mat
# array type: 3x9x200(3 eigenvectors x 9 regions x 200 nodes)

### we will get degree centralities of subcortical-weighted manifolds in terms for 9 regions

In [ ]:
for i in range(0,8+1):
    sw_mean = np.zeros((100,3))
    for j in range(1,100+1):
        sw_manifold = getattr(mod,'sw_manifold{}'.format(j))
        row = sw_manifold[:,i,:]
        rmean = row.mean(axis=1)
        sw_mean[j-1,:] = rmean
    globals()['sw_mean{}'.format(str(i+1))] = sw_mean

concatenate 9 sw_means

In [ ]:
sw_means = np.concatenate([getattr(mod, f'sw_mean{i}') for i in range(1,9+1)],1)

In [ ]:
col1 = [f'sw_mani1_{i}' for i in range(1,9+1)]
col2 = [f'sw_mani2_{i}' for i in range(1,9+1)]
col3 = [f'sw_mani3_{i}' for i in range(1,9+1)]

In [ ]:
col1.extend(col2)

In [ ]:
col1.extend(col3)

In [ ]:
sw_means = pd.DataFrame(sw_means)

In [ ]:
sw_means.columns = col1

In [ ]:
sw_means = sw_means[:50]

In [ ]:
X_dat_s = sw_means

### regress out

1. we make linear model which has X = rm and y = one column among all features
2. we then get predicted value by putting X into the model again

> lm.fit(X, y)

In [ ]:
# 27 sw_manifold terms
for i in range(0,26+1):
    lm = LinearRegression()
    lm.fit(rm, sw_means.iloc[:,i])
    pred = lm.predict(rm)
    globals()[f'sw_means{i+1}'] = pred.reshape(NumSubj,1)

In [ ]:
sw_means = np.concatenate([getattr(mod, f'sw_means{i}') for i in range(1,27+1)],1)

In [ ]:
grad_s = sw_means

we remove X_dat from grad to get corrected terms

In [ ]:
X_grad_s = X_dat_s - grad_s

we set y

In [ ]:
y = pd.DataFrame(ha_freq)

In [ ]:
y.columns = ['ha_freq']

we make one model that concatenate X_grad and X_grad_s

In [ ]:
from statsmodels.formula.api import ols
import matplotlib
from __future__ import print_function
import statsmodels.api as sm
import matplotlib.pyplot as plot
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
X_grad_rst = pd.concat([X_grad,X_grad_s],axis=1)

In [ ]:
matplotlib.rcParams['axes.unicode_minus'] = False

we compute ols with X_grad_rst and ha_freq

In [ ]:
# summary and modeling
model = sm.OLS(y, X_grad_rst)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

## <span style="color:#2D3748; background-color:#fff5b1;">supervised machine learning</span>

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error as mse


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

from math import isclose

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
from math import sqrt
import sklearn

from sklearn.model_selection import RepeatedKFold
import time

### generate outer fold 

In [ ]:
alphas = np.array((0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1))

In [ ]:
rfk = RepeatedKFold(n_splits=5, n_repeats=100,random_state=1219)

### 100 trials loop

- get the set of indices using 100 trials loop

In [ ]:
i = 0
j = 0
train = []
test = []
for train_index, test_index in rfk.split(X_grad_rst):
    i = i + 1
    train.append(train_index)
    test.append(test_index)
    if i % 5 == 0:
        j = j+1
        globals()['train{}'.format(j)] = train 
        globals()['test{}'.format(j)] = test
        train = []
        test = []

- get the datasets of outer folds at each trial

In [ ]:
for i in range(1,100+1):
    train_ = getattr(mod, 'train{}'.format(i))
    test_ = getattr(mod, 'test{}'.format(i))
    
    for out in range(1,5+1):
        train_idx = train_[out-1]
        test_idx = test_[out-1]

        # e.g., X_train1_1: 1st trial's 1st outer fold's dataset
        globals()['X_train{}_{}'.format(i,out)] = X_grad_rst.iloc[train_idx]
        globals()['X_test{}_{}'.format(i,out)] = X_grad_rst.iloc[test_idx]
        globals()['y_train{}_{}'.format(i,out)] = y.iloc[train_idx]
        globals()['y_test{}_{}'.format(i,out)] = y.iloc[test_idx]

- perform inner fold loop and alpha loop at each outer fold

In [ ]:
start = time.time()
for i in range(1,100+1):
    print('\n\033[1m'+'### {}th tiral ###'.format(i)+'\033[0m')
    for out in range(1,5+1):
        print('·outfold={}'.format(out))
        X_train_ = getattr(mod,'X_train{}_{}'.format(i,out))
        X_test_ = getattr(mod,'X_test{}_{}'.format(i,out))
        y_train_ = getattr(mod,'y_train{}_{}'.format(i,out))
        y_test_ = getattr(mod,'y_test{}_{}'.format(i,out))

        
        np.random.seed(0)
        kf = KFold(n_splits=5,shuffle=True)
        inner = 0
        inner_icc, inner_mae = [], []
        for inner_train, inner_test in kf.split(X_train_):
            inner = inner + 1
            print(' - inner_fold={}'.format(inner))
            x_train_sbt, x_val = X_train_.iloc[inner_train], X_train_.iloc[inner_test]
            y_train_sbt, y_val = y_train_.iloc[inner_train], y_train_.iloc[inner_test]
            
            alpha_mae = []
            alpha_icc = []
            for alpha in range(1,10+1):
                np.random.seed(0)
                lasso = Lasso(alpha=alphas[alpha-1])
                
                lasso.fit(x_train_sbt,y_train_sbt)
                
                # save indices of selected features through lasso
                lasso_idx = np.where(lasso.coef_!=0)
                
                # if there is no selected feature, we put weird values,
                # and then interrupt the corresponding iteration and pass the next iteration.
                if np.sum(lasso_idx) == 0:
                    alpha_mae.append(100)
                    alpha_icc.append(-100)
                    continue
                
                # if there is selected feature, we perform next working
                globals()['alpha_lasso{}'.format(alpha)] = lasso_idx
                # save feature names with corresponding alpha iteration
                
                # we make linear model only using selected features
                selected_col = x_train_sbt.columns[lasso_idx]
                
                x_train_sel = x_train_sbt[selected_col]
                x_val_sel = x_val[selected_col]
                
                lm = LinearRegression()
                lm.fit(x_train_sel, y_train_sbt)
                globals()['alpha_model{}'.format(alpha)] = lm
                # we save the model's name with corresponding alpha iteration number together
                
                pred = lm.predict(x_val_sel)
                mae = mean_absolute_error(y_val, pred)
                
                # icc computation
                dat = pd.concat([y_val.reset_index(drop=True), pd.DataFrame(pred)], axis=1)
                ns = len(dat) # row number
                nr = len(dat.columns) # col number
                MSr = dat.mean(axis=1).var() * nr
                MSw = np.sum(dat.var(axis=1) / ns)
                icc = (MSr - MSw) / (MSr + (nr-1)*MSw)
                
                alpha_mae.append(mae)
                alpha_icc.append(icc)
            
            # we got out of the alpha loop
            # now, it is inner loop
            
            # Based on alpha loop, we save the best model and the lasso indices of the model with accuracy
            
            # But, since icc may overlap, we perform the following
            # also, if icc has -100 values in all alphas, 
            # we do not save the model corresponding the inner_fold
            
            # maximum value of icc
            icc = np.max(alpha_icc)
            
            # index of maximum value of icc
            num = np.where(alpha_icc == icc)[0]

            
            # when the maximum of icc has more than two values,
            # it will return the smaller of them.
            # but, even if there is one maximum value, it applies also.
            alpha_mae = pd.DataFrame(alpha_mae)
            mae = alpha_mae.iloc[num].min()[0]
            
            # we save index to select the best model based on the icc
            # if icc and mae are same, save as first index.
            alpha_idx = alpha_mae.index[alpha_mae[0] == mae][0]
            
            alpha_idx = alpha_idx + 1
                
            ### now, we select the best model in alpha loop based on accuracy,
            ### and then save it as the model corresponding inner fold iteration 
            globals()['model{}_{}_{}'.format(i,out,inner)] = getattr(mod,'alpha_model{}'.format(alpha_idx))
            globals()['lasso{}_{}_{}'.format(i,out,inner)] = getattr(mod,'alpha_lasso{}'.format(alpha_idx))
            
            # also save accuracy
            globals()['mae{}_{}_{}'.format(i,out,inner)] = mae
            globals()['icc{}_{}_{}'.format(i,out,inner)] = icc
end = time.time()
print('Done!')
print('time:', end - start)

- we set the best model of inner fold as outer fold model

In [ ]:
for i in range(1,100+1):
    for out in range(1,5+1):
        inner_mae = []
        inner_icc = []
        for inner in range(1,5+1):
            mae_ = getattr(mod,'mae{}_{}_{}'.format(i,out,inner))
            icc_ = getattr(mod,'icc{}_{}_{}'.format(i,out,inner))
            
            globals()['inner_model{}'.format(inner)] = getattr(mod,'model{}_{}_{}'.format(i,out,inner)) 
            globals()['inner_lasso{}'.format(inner)] = getattr(mod,'lasso{}_{}_{}'.format(i,out,inner))

            inner_mae.append(mae_)
            inner_icc.append(icc_)
        
        # we got out of the inner fold loop
        # now it is outer loop
        
        # we select the best model in inner fold loop

        # maximum value of icc
        icc = np.max(inner_icc)

        # index of maximum value of icc
        num = np.where(inner_icc == icc)[0]
        
        # when the maximum of icc has more than two values,
        # it will return the smaller of them.
        # but, even if there is one maximum value, it applies also.
        inner_mae = pd.DataFrame(inner_mae)
        mae = inner_mae.iloc[num].min()[0]
        
        
        # we save index to select the best model based on the icc
        # if icc and mae are same, save as first index.
        inner_idx = inner_mae.index[inner_mae[0] == mae][0]
        
        inner_idx = inner_idx + 1

        ### now, we select the best model in inner fold based on accuracy,
        ### and then save it as the model corresponding outer fold iteration 
        globals()['model{}_{}'.format(i,out)] = getattr(mod,'inner_model{}'.format(inner_idx))
        globals()['lasso{}_{}'.format(i,out)] = getattr(mod,'inner_lasso{}'.format(inner_idx))

        # also save accuracy
        globals()['mae{}_{}'.format(i,out)] = mae
        globals()['icc{}_{}'.format(i,out)] = icc

- set lasso indices of the best model at each outer fold as finally selected lasso indices

In [ ]:
for i in range(1,100+1):
    outer_mae = []
    outer_icc = []
    for out in range(1,5+1):
        mae_ = getattr(mod,'mae{}_{}'.format(i,out))
        icc_ = getattr(mod,'icc{}_{}'.format(i,out))

        globals()['outer_model{}'.format(out)] = getattr(mod,'model{}_{}'.format(i,out)) 
        globals()['outer_lasso{}'.format(out)] = getattr(mod,'lasso{}_{}'.format(i,out))

        outer_mae.append(mae_)
        outer_icc.append(icc_)

    # we got out of the outer fold loop
    # now it is trial loop


    # we select the best model in outer fold loop


    # maximum value of icc
    icc = np.max(outer_icc)

    # index of maximum value of icc
    num = np.where(outer_icc == icc)[0]

    # when the maximum of icc has more than two values,
    # it will return the smaller of them.
    # but, even if there is one maximum value, it applies also.
    outer_mae = pd.DataFrame(outer_mae)
    mae = outer_mae.iloc[num].min()[0]

    # we save index to select the best model based on the icc
    # if icc and mae are same, save as first index.
    outer_idx = outer_mae.index[outer_mae[0] == mae][0]


    outer_idx = outer_idx + 1

    ### now, we select the best model in outer fold based on accuracy,
    ### and then save its lasso indices corresponding the outer fold iteration 
    globals()['final_lasso{}'.format(i)] = getattr(mod,'outer_lasso{}'.format(outer_idx))

    # acc도 저장
    globals()['mae{}'.format(i)] = mae
    globals()['icc{}'.format(i)] = icc

#### we will perform 5-fold cross validations based on existing nested cross-validation results
- we will save the results from this for later analysis

1. put best selected features of outer fold defined above,
2. perform 5-fold cross validation with 50 subjects

In [ ]:
for i in range(1,100+1):
    lasso_ = getattr(mod,'final_lasso{}'.format(i))
    
    # select column using lasso feature selection
    col_selected = X_grad_rst.columns[lasso_]
    
    X_selected = X_grad_rst[col_selected]

    pred_final = np.array(())
    y_final = np.array(())

    np.random.seed(0)
    kf = KFold(n_splits=5, shuffle=True)

    for train_index, test_index in kf.split(X_selected):
        x_train, x_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        lm = LinearRegression()
        lm.fit(x_train, y_train)
        pred = lm.predict(x_test)
        
        pred_final = np.append(pred_final, pred)
        y_final = np.append(y_final, y_test)
    
    # after finishing final 5-fold cross validation, 
    # save predicted and actual values for each trial
    globals()['pred_final{}'.format(i)] = pred_final
    globals()['y_final{}'.format(i)] = y_final

plotting it into brain surface

In [ ]:
import nibabel as nib
from nilearn.datasets import fetch_surf_fsaverage

import brainstat
from brainstat.stats.terms import FixedEffect

from scipy import io

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

import enigmatoolbox
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical
from enigmatoolbox.plotting.surface_plotting import plot_subcortical
from mlxtend.evaluate import permutation_test
import scipy.stats as st

In [ ]:
# get brainstat dir
brainstat_dir = os.path.dirname(brainstat.__file__)

In [ ]:
labeling = load_parcellation('schaefer', scale=200, join=True)
mask = labeling != 0

# generate labeing and mask based on schaefer200

surf_lh, surf_rh = load_conte69()

In [ ]:
ctx_surface =  np.zeros((200,3))
sctx_surface = np.zeros((9,3))

In [ ]:
# continuously append indices of selected features durnig 100 trials
for i in range(1,100+1):
    final_lasso = getattr(mod,'final_lasso{}'.format(i))
    final_lasso = final_lasso[0]-1 
    
    full = np.ones((627,))
    empty = np.zeros((627,))
    empty[final_lasso] = full[final_lasso]

    ctx = np.zeros((200,3))
    sctx = np.zeros((9,3))
    

    ctx[:,0] = empty[0:200,]
    ctx[:,1] = empty[200:400,]
    ctx[:,2] = empty[400:600,]
    
    sctx[:,0] = empty[600:609,]
    sctx[:,1] = empty[609:618,]
    sctx[:,2] = empty[618:627,]

    
    ctx_surface = ctx_surface + ctx
    sctx_surface = sctx_surface + sctx

ctx_surface = ctx_surface/100
sctx_surface = sctx_surface/100

### ctx_surface

ctx_surface_sum

In [ ]:
ctx_surface_sum = np.sum(ctx_surface,axis=1).round(2).reshape(200,1)

In [ ]:
num_max = np.max(ctx_surface_sum)

In [ ]:
num_max

In [ ]:
gradients_kernel = [None] * 1 

gradients_kernel[0] = map_to_labels(ctx_surface_sum[:, 0], labeling, mask=mask,
                                       fill=np.nan)

    
label_text = ['']
# plot_hemispheres(surf_lh, surf_rh, array_name=gradients_kernel, size=(1200,300),
#                 cmap='Reds', color_bar=True,label_text=label_text, zoom=1,color_range=(0,num_max)) 
plot_hemispheres(surf_lh, surf_rh, array_name=gradients_kernel, size=(1200,300),
                cmap='Reds', color_bar=False,label_text=label_text, zoom=1,color_range=(0,0.8), screenshot=True, filename='./fig4/ha_freq/revised_ctx.png') 

### sctx_surface

sctx_surface_sum

In [ ]:
sctx_surface_sum = np.sum(sctx_surface,axis=1).round(2).reshape(9,1)

In [ ]:
sctx_surface_sum = np.concatenate([sctx_surface_sum[0:7],sctx_surface_sum[0:7]]).reshape(14,)

In [ ]:
num_max_s = np.max(sctx_surface_sum)

In [ ]:
num_max_s

In [ ]:
# plot_subcortical(array_name=sctx_surface, ventricles=False, size=(800, 400),
#                  cmap='Reds', color_range=(0,1), color_bar=True)
plot_subcortical(array_name=sctx_surface_sum, ventricles=False, size=(800, 400),
                 cmap='Reds', color_range=(0,num_max_s), color_bar=False, screenshot=True, filename='./fig4/ha_freq/revised_sctx.png')

### report the accuracy of the 100 trials

In [ ]:
mae_list = []
rmse_list = []
icc_list = []
corr_list = []
# after importing the actual and predicted values for the existing 100 trials,
# we will report the accuracy values
for i in range(1,100+1):
    y = getattr(mod,'y_final{}'.format(i))
    pred = getattr(mod,'pred_final{}'.format(i))
    
    mae = mean_absolute_error(y,pred)
    rmse = sqrt(mean_squared_error(y,pred))
    
    # icc
    dat = pd.concat([pd.DataFrame(y),pd.DataFrame(pred)],axis=1)

    # row number
    ns = len(dat)
    # col number
    nr = len(dat.columns)

    MSr = dat.mean(axis=1).var() * nr
    MSw = np.sum(dat.var(axis=1) / ns)

    icc = (MSr - MSw)/(MSr + (nr - 1) * MSw) ## 이게 icc

    corr = pearsonr(y,pred)[0]
        
    mae_list.append(mae)
    rmse_list.append(rmse)
    icc_list.append(icc)
    corr_list.append(corr)

# #create 95% confidence interval for population mean weight
# st.t.interval(alpha=0.95, df=len(mae_list)-1, loc=np.mean(mae_list), scale=st.sem(mae_list)) 

mean_mae = np.mean(mae_list).round(2)
mean_rmse = np.mean(rmse_list).round(2)
mean_icc = np.mean(icc_list).round(2)
mean_corr = np.mean(corr_list).round(2)

std_mae = np.std(mae_list).round(2)
std_rmse = np.std(rmse_list).round(2)
std_icc = np.std(icc_list).round(2)
std_corr = np.std(corr_list).round(2)    

L_mae, U_mae = (mean_mae-std_mae).round(2),(mean_mae+std_mae).round(2)
L_rmse, U_rmse = (mean_rmse-std_rmse).round(2),(mean_rmse+std_rmse).round(2)
L_icc, U_icc = (mean_icc-std_icc).round(2),(mean_icc+std_icc).round(2)
L_corr, U_corr = (mean_corr-std_corr).round(2),(mean_corr+std_corr).round(2)



print('mae:({},{})'.format(L_mae, U_mae))
print('rmse:({},{})'.format(L_rmse, U_rmse))
print('icc:({},{})'.format(L_icc, U_icc))
print('r:({},{})'.format(L_corr, U_corr))

#### permutation test between actual and predicted values

In [ ]:
import random

In [ ]:
perm_p_list = []
for i in range(1,100+1):
    y = getattr(mod,'y_final{}'.format(i))
    pred = getattr(mod,'pred_final{}'.format(i))

    actual_corr = pearsonr(y,pred)[0]
    

    i = 1
    perm_r_list = []
    
    # repeat the loop until i = 1000
    while i < 1000+1:
        i = i+1
        
        #1. shuffle 50 indices randomly
        perm_idx = random.sample(range(0,49+1),k=50)
        #2. put mixed indices into y, and the select y randomly
        y_perm = y[perm_idx]
        #3. compute the correlation between mixed y and predicted y
        r_perm = np.corrcoef(y_perm,pred)[0][1]
        #4. store the correlation coefficients in a perm_r_list one by one
        # repeat this for 1000 times
        perm_r_list.append(r_perm)

    # perform the following actions after 1000 trials
    perm_r_list = np.array(perm_r_list)
    #5. store the number of values greater than the actual correlation coefficients in the perm_r_list
    n_perm = len(perm_r_list[perm_r_list>actual_corr])
    #6. divide the number of actual values by 1000.
    # that is the p-value
    p_perm = n_perm/1000
    #7. p-value obtained in this way is stored as p-value of nth trial one by one
    perm_p_list.append(p_perm)

In [ ]:
mean_pval = np.round(np.mean(perm_p_list),2)

In [ ]:
print('r = {}±{}, p = {} (ICC = {}±{}, MAE = {}±{})'.format(mean_corr,std_corr,mean_pval,mean_icc,std_icc,mean_mae,std_mae))

### scatter plot of the averages of 100 iterations

In [ ]:
y_sum = np.zeros((50,))
pred_sum = np.zeros((50,))

In [ ]:
for i in range(1,100+1):
    y = getattr(mod,'y_final{}'.format(i))
    pred = getattr(mod,'pred_final{}'.format(i))
    
    y_sum = y_sum + y
    pred_sum = pred_sum + pred
y_mean = y_sum/100
pred_mean = pred_sum/100

In [ ]:
final = pd.DataFrame([y_mean,pred_mean]).T
final.columns = ['actual_y','predicted_y']

In [ ]:
fig, ax = plt.subplots()

ax.scatter(y_mean,pred_mean,color='grey')
ax.set_aspect(0.5/ax.get_data_ratio(), adjustable='box')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

sns.regplot(x='actual_y',y='predicted_y',data=final, scatter=False, color='black')

plt.xlabel('')
plt.ylabel('')

plt.xlim(0,14)
plt.ylim(0,14)

# ax.axes.xaxis.set_ticklabels([0,2,4,6,8,10,12,14])

plt.savefig('./fig4/ha_freq/scatter_plot_revised.png')